In [1]:
%config Completer.use_jedi = False

# 1. 파이토치 제공 데이터 사용

In [2]:
import torch
import torchvision
import torchvision.transforms as tr
from torch.utils.data import DataLoader, Dataset
import numpy as np

# 아래 데이터를 다운로드 받을 때 들어오는 데이터들에 대한 전처리를 어떻게 할 것인지...
# 하고 싶은 전처리 작업들을 Compose() 안의 리스트[]에다가 나열해주면 된다.
transf = tr.Compose([tr.Resize(8), tr.ToTensor()]) # 8x8로 리사이즈, 텐서데이터로 변환

trainset = torchvision.datasets.CIFAR10(root='./cifardata', train=True, download=True, transform=transf)
testset = torchvision.datasets.CIFAR10(root='./cifardata', train=False, download=True, transform=transf) 

Files already downloaded and verified
Files already downloaded and verified


In [3]:
trainset[0][0].size()

torch.Size([3, 8, 8])

In [4]:
trainloader = DataLoader(trainset, batch_size=50, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=50, shuffle=True, num_workers=2)

In [6]:
len(trainloader) # CIFAR10 데이터셋의 train 데이터가 5만장인데, batch를 50으로 잡았으니 당연히 1000번 돌게 된다.

1000

# 2. (개인 데이터를 이용하는 방법) 같은 클래스 별 폴더 이미지 데이터 이용

In [9]:
# 깔끔하게 클래스를 나눠서 폴더 이미지를 나눠놓으면 학습하기 편하다. 예를 들어...
# ./class/yl_bottles ./class/nl_bottles

transf = tr.Compose([tr.Resize(16), tr.ToTensor()])
trainset = torchvision.datasets.ImageFolder(root='./data/sample_train/', transform=transf) 
# ImageFolder라는 함수는 알아서 해당 경로 안의 디렉토리들을 탐색해준다.
# 그리고 각각의 폴더에 대해서 자동으로 레이블링을 다르게 매겨준다.
# 그리고 추가적으로 여기서 바로 전처리로 transform을 먹일 수 있다. 
# 즉 이 한줄로 데이터를 전부 불러오면서, 레이블링도 해주면서, 전처리까지 한번에 할 수 있다는 것이다.
trainloader = DataLoader(trainset, batch_size=2, shuffle=False, num_workers=2)

In [11]:
print(len(trainloader)) # 현재 가지고 있는 데이터가 총 28장이기 때문에 batch사이즈를 2로 주니까, DataLoader는 당연히 14번 돌게 된다.
trainset[0][0].size()

14


torch.Size([3, 21, 16])

# 3. 개인 데이터 사용 (2 types)

In [13]:
# import processing
train_images = np.random.randint(256, size=(20, 32, 32, 3)) # 넘파이로 들어온 이미지가 있다고 가정하겠다.(32x32 3채널 이미지 20개)
train_labels = np.random.randint(2, size=(20, 1))

# 데이터는 이미 우리가 np.text라던가 pandas라던가 pil images라던가 opencv라던가 다양한 함수로 받았다고 가정하고, 그것을 .........
print(train_images.shape, train_labels.shape)

(20, 32, 32, 3) (20, 1)


In [14]:
# 개인 데이터를 다룰 때 이런 클래스 형태는 거의 정형화 되어있다.
class TensorData(Dataset):
    def __init__(self, a_data, b_data):
        self.a_data = torch.FloatTensor(a_data)
        self.a_data = self.a_data.permute(0, 3, 1, 2) # 위에서 들어온 (20, 32, 32, 3)의 자료형 순서를 (20, 3, 32, 32)로 바꿔주겠다는 뜻
        self.b_data = torch.LongTensor(b_data)
        self.len = self.b_data.shape[0]
        
    def __getitem__(self, index):
        return self.a_data[index], self.b_data[index]

    def __len__(self):
        return self.len

In [15]:
train_data = TensorData(train_images, train_labels)
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)

In [17]:
print(len(train_images))
train_data[0][0].size()

20


torch.Size([3, 32, 32])

In [31]:
data_iter = iter(train_loader)
images, labels = next(data_iter)

In [32]:
images

tensor([[[[180., 131., 229.,  ..., 178., 207., 186.],
          [159.,  48., 142.,  ..., 176., 142.,  47.],
          [247.,  72.,  67.,  ..., 139.,  99., 125.],
          ...,
          [177., 192., 115.,  ..., 245., 121.,  93.],
          [ 14.,  36., 218.,  ..., 154.,  92., 150.],
          [246.,  14.,  76.,  ...,  94.,  51.,  59.]],

         [[ 58., 114., 247.,  ..., 112.,  35., 163.],
          [105., 103.,  90.,  ..., 183., 163., 140.],
          [103.,  32., 121.,  ..., 226.,   3., 206.],
          ...,
          [233.,  56.,  96.,  ...,  53., 177.,   2.],
          [ 10.,  84., 231.,  ..., 137., 231., 232.],
          [ 49., 172., 121.,  ..., 222., 137., 136.]],

         [[ 94., 244., 137.,  ..., 195., 119.,  59.],
          [181., 181., 226.,  ...,   9., 139.,  12.],
          [136., 226., 212.,  ..., 114.,  84.,  88.],
          ...,
          [114.,  36.,  49.,  ...,  98.,  12., 253.],
          [ 69., 152., 224.,  ..., 129., 131.,  24.],
          [ 31., 130., 180.,  ...

In [19]:
################################################################################################v

In [21]:
class MyDataset(Dataset):
    def __init__(self, image_data, label_data, transform=None):
        self.image_data = image_data
        self.label_data = label_data
        self.transform = transform
        self.len = len(label_data)

    def __getitem__(self, index):
        sample = self.image_data[index], self.label_data[index]

        if self.transform:
            sample = self.transform(sample)

        return sample

    def __len__(self):
        return self.len


class ToTensor:
    def __call__(self, sample):
        inputs, labels = sample
        inputs = torch.FloatTensor(inputs)
        inputs = inputs.permute(2, 0, 1)
        return inputs, torch.LongTensor(labels)


class LinearTensor:
    def __init__(self, slope=1, bias=0):
        self.slope = slope
        self.bias = bias

    def __call__(self, sample):
        inputs, labels = sample
        inputs = self.slope * inputs + self.bias
        return inputs, labels

In [22]:
# 사용자가 직접 만든 전처리 함수를 파이토치 컴포즈에 넣을 수 있다!!!
trans = tr.Compose([ToTensor(), LinearTensor(2, 5)])
dataset1 = MyDataset(train_images, train_labels, transform=trans)
train_loader1 = DataLoader(dataset1, batch_size=10, shuffle=True)

In [24]:
# 위에서 만든 사용자 함수를 통해서 최초에 opencv같은 함수들을 통해서 np로 들어온 이미지가 최종적으로 torch.Tensor로 변환된 것을 확인할 수 있다.
first_data = dataset1[0]
features, labels = first_data
print(type(features), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [28]:
dataiter1 = iter(train_loader1)
images1, labels1 = next(dataiter1)

In [29]:
images1

tensor([[[[289., 381., 307.,  ...,  25., 309., 313.],
          [123., 187., 367.,  ...,   5., 401., 115.],
          [147., 487., 375.,  ..., 319., 485., 459.],
          ...,
          [231., 283., 295.,  ..., 455., 381., 145.],
          [339., 363., 127.,  ..., 107., 167., 507.],
          [397., 363., 397.,  ..., 375., 253., 325.]],

         [[323., 175., 283.,  ..., 237., 469., 263.],
          [345., 249.,  93.,  ...,  71.,   5., 479.],
          [459., 177., 389.,  ..., 421.,  93., 303.],
          ...,
          [311., 279.,  95.,  ...,  43., 249.,  21.],
          [311., 387.,  11.,  ..., 497., 453., 201.],
          [175., 203., 205.,  ..., 377., 429., 123.]],

         [[ 59., 437., 243.,  ..., 101., 395., 257.],
          [297., 245.,  93.,  ..., 185.,  65., 481.],
          [169., 411., 331.,  ..., 433., 453., 147.],
          ...,
          [315., 173.,  41.,  ..., 323., 161.,  13.],
          [113., 203., 205.,  ..., 105., 323., 197.],
          [105.,  91., 191.,  ...

In [33]:
####################################################################################

In [37]:
class MyDataset(Dataset):
    def __init__(self, image_data, label_data, transform=None):
        self.image_data = image_data
        self.label_data = label_data
        self.transform = transform
        self.len = len(label_data)

    def __getitem__(self, index):
        sample = self.image_data[index], self.label_data[index]

        if self.transform:
            sample = self.transform(sample)

        return sample

    def __len__(self):
        return self.len


class MyTransform:
    def __call__(self, sample):
        inputs, labels = sample
        inputs = torch.FloatTensor(inputs)
        inputs = inputs.permute(2, 0, 1)
        labels = torch.FloatTensor(labels)

        transf = tr.Compose([tr.ToPILImage(), tr.Resize(128), tr.ToTensor(), tr.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        final_output = transf(inputs)

        return final_output, labels

In [38]:
ds2 = MyDataset(train_images, train_labels, transform=MyTransform())
train_loader2 = DataLoader(ds2, batch_size=10, shuffle=True)

In [39]:
ffirst_data = ds2[0]
features, labels = ffirst_data
print(type(features), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [40]:
dataiter2 = iter(train_loader2)
image2, label2 = next(dataiter2)

In [41]:
print(image2.shape)

torch.Size([10, 3, 128, 128])


In [42]:
image2

tensor([[[[-0.7176, -0.7176, -0.7176,  ..., -0.4745, -0.6314, -0.6314],
          [-0.7176, -0.7176, -0.7176,  ..., -0.4745, -0.6314, -0.6314],
          [-0.5137, -0.5137, -0.5373,  ..., -0.4196, -0.5608, -0.5608],
          ...,
          [ 0.2549,  0.2549,  0.2471,  ...,  0.1765,  0.2941,  0.2941],
          [ 0.4353,  0.4353,  0.4039,  ...,  0.2235,  0.3569,  0.3569],
          [ 0.4353,  0.4353,  0.4039,  ...,  0.2235,  0.3569,  0.3569]],

         [[-0.5216, -0.5216, -0.4980,  ..., -0.4196, -0.4275, -0.4275],
          [-0.5216, -0.5216, -0.4980,  ..., -0.4196, -0.4275, -0.4275],
          [-0.5216, -0.5216, -0.4980,  ..., -0.4510, -0.4588, -0.4588],
          ...,
          [ 0.4980,  0.4980,  0.4275,  ..., -0.5843, -0.5843, -0.5843],
          [ 0.7020,  0.7020,  0.6235,  ..., -0.6314, -0.6314, -0.6314],
          [ 0.7020,  0.7020,  0.6235,  ..., -0.6314, -0.6314, -0.6314]],

         [[-0.6784, -0.6784, -0.5608,  ..., -0.9451, -0.9765, -0.9765],
          [-0.6784, -0.6784, -